In [1]:
!pip install category_encoders

     |████████████████████████████████| 102kB 3.5MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.dummy import DummyRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GaussianNoise, GaussianDropout
from tensorflow.keras import optimizers
from sklearn.preprocessing import StandardScaler

In [0]:
age19t = pd.read_csv('Age19-seasons.csv')
age20t = pd.read_csv('Age20-seasons.csv')
age21t = pd.read_csv('Age21-seasons.csv')
age22t = pd.read_csv('Age22-seasons.csv')
age23t = pd.read_csv('Age23-seasons.csv')
age24t = pd.read_csv('Age24-seasons.csv')
age25t = pd.read_csv('Age25-seasons.csv')
age26t = pd.read_csv('Age26-seasons.csv')
age27t = pd.read_csv('Age27-seasons.csv')
age28t = pd.read_csv('Age28-seasons.csv')
age29t = pd.read_csv('Age29-seasons.csv')
age30t = pd.read_csv('Age30-seasons.csv')
age31t = pd.read_csv('Age31-seasons.csv')

age19f = pd.read_csv('age19f.csv')
age20f = pd.read_csv('age20f.csv')
age21f = pd.read_csv('age21f.csv')
age22f = pd.read_csv('age22f.csv')
age23f = pd.read_csv('age23f.csv')
age24f = pd.read_csv('age24f.csv')
age25f = pd.read_csv('age25f.csv')
age26f = pd.read_csv('age26f.csv')
age27f = pd.read_csv('age27f.csv')
age28f = pd.read_csv('age28f.csv')
age29f = pd.read_csv('age29f.csv')
age30f = pd.read_csv('age30f.csv')
age31f = pd.read_csv('age31f.csv')

In [0]:
targets_dict = {19:age19t, 20:age20t, 21:age21t, 22:age22t, 23:age23t, 24:age24t,
            25:age25t, 26:age26t, 27:age27t, 28:age28t, 29:age29t, 30:age30t,
            31:age31t}
features_dict = {19:age19f, 20:age20f, 21:age21f, 22:age22f, 23:age23f, 24:age24f,
            25:age25f, 26:age26f, 27:age27f, 28:age28f, 29:age29f, 30:age30f,
            31:age31f}

for i in range(19,32):
    features_dict[i] = features_dict[i].add_suffix('_age_' + str(i))
    features_dict[i] = features_dict[i].rename(columns={"playerid_age_" + str(i): "playerid"})

for i in range(19,32):
    targets_dict[i] = targets_dict[i].drop(columns=['Name', 'Season', 'Team'])
    targets_dict[i] = targets_dict[i].add_suffix('_age_' + str(i))
    targets_dict[i] = targets_dict[i].rename(columns={"playerid_age_" + str(i): "playerid"})

In [0]:
class Models:

    def __init__(self, playername, age_season, features, targets):
        self.playername = playername
        self.age_season = age_season
        self.features = features
        self.targets = targets


    def wrangle(self):
        df_list = []
        for i in range(self.age_season - 3, self.age_season):
            df_list.append(self.features[i])
            df_list.append(self.targets[i])

        targets_df = self.targets[self.age_season]
        df_list.append(targets_df)

        parent_df = reduce(lambda left,right: pd.merge(left,right,on=['playerid'],
                                                    how='outer'), df_list)

        colstocheck = parent_df.columns
        parent_df[colstocheck] = parent_df[colstocheck].replace({'%': ''}, regex=True)

        prior_year = self.age_season - 1

        train = parent_df[parent_df['Name_age_' + str(prior_year)] != self.playername]
        test = parent_df[parent_df['Name_age_' + str(prior_year)] == self.playername]
        #train, test = train_test_split(parent_df, random_state=33, train_size=.99)

        targets = targets_df[targets_df.columns[~targets_df.columns.str.contains('playerid')]].columns
        features = parent_df[parent_df.columns[~parent_df.columns.str.contains('Season|Name|playerid')]]
        features = features.drop(columns=targets).columns
        self.targets = targets

        train = train.dropna(subset=targets)
        test = test.dropna(subset=targets)

        X_train = train[features]
        Y_train = train[targets]
        X_test = test[features]
        Y_test = test[targets]

        cols = X_train.columns.drop(X_train.columns[0])
        X_train.loc[:,cols] = X_train[cols].apply(pd.to_numeric, errors='coerce')

        X_test.loc[:,cols] = X_test[cols].apply(pd.to_numeric, errors='coerce')

        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.Y_test = Y_test


    def PMAPP_LR(self):
        pipeline0 = make_pipeline(
            ce.OneHotEncoder(),
            SimpleImputer(strategy='mean'),
            LinearRegression(n_jobs=-1)
        )
        pipeline0.fit(self.X_train, self.Y_train)
        pred0 = pipeline0.predict(self.X_test)
        self.pred0 = pred0


    def PMAPP_RFR(self):
        pipeline1 = make_pipeline(
            ce.OneHotEncoder(),
            SimpleImputer(strategy='mean'),
            RandomForestRegressor(n_estimators=800, max_features=.15,
            random_state=99, n_jobs=-1)
        )
        pipeline1.fit(self.X_train, self.Y_train)
        pred1 = pipeline1.predict(self.X_test)
        self.pred1 = pred1
        
        
    def PMAPP_NN(self, epochs=200):
        X_train = self.X_train
        Y_train = self.Y_train
        X_test = self.X_test
        
        ohe = ce.OneHotEncoder()
        ohe.fit(X_train)
        X_train_enc = ohe.transform(X_train)
        X_train_imp = SimpleImputer(strategy='most_frequent').fit_transform(X_train_enc)
        ssX = StandardScaler()
        X_train_scaled = ssX.fit_transform(X_train_imp)
        
        X_test_enc = ohe.transform(X_test)
        X_test_imp = SimpleImputer(strategy='most_frequent').fit_transform(X_test_enc)
        X_test_scaled = ssX.transform(X_test_imp)
        
        ssY = StandardScaler()
        Y_train_scaled = ssY.fit_transform(Y_train)

        model = Sequential()
        model.add(Dense(328, input_dim=X_train_scaled.shape[1], activation='linear'))
        model.add(Dense(1000, activation='linear'))
        model.add(Dense(1000, activation='linear'))
        model.add(Dropout(0.5))
        model.add(Dense(1000, activation='linear'))
        model.add(Dense(19, activation='linear'))

        sgd = optimizers.SGD(learning_rate=0.07, momentum=0.1, nesterov=False)

        model.compile(loss='logcosh', optimizer=sgd, metrics=['mape', 'mae'])
        model.fit(X_train_scaled, Y_train_scaled, epochs=epochs, validation_split=0.04,
                  shuffle=True)

        self.pred2 = ssY.inverse_transform(model.predict(X_test_scaled))


    def Dummy(self):
        Dumb = DummyRegressor(strategy='mean')
        Dumb.fit(self.X_train, self.Y_train)
        pred3 = Dumb.predict(self.X_test)
        self.pred3 = r2_score(self.Y_test, pred3, multioutput='raw_values')


    def show(self):
        #token = []
        #print(self.playername)
        np.set_printoptions(suppress=True)
        
        try:
            print('Linear Regression:', self.pred0)
        
        except:
            pass

        try:
            print('Random Forest:', self.pred1)

        except:
            pass

        try:
            print('Neural Network:', self.pred2)

        except:
            pass

        try:
            print('Dummy:', self.pred3)
      
        except:
            pass
        print(self.targets)
        #for i in range(19):
            #token.append(format(self.pred1[0][i], '.3f'))
        """
            elif i < 6:
                token.append(format(self.pred0[0][i], '.0f'))
            elif i < 8:
                token.append(format(self.pred1[0][i], '.1f'))
            elif i < 14:
                token.append(format(self.pred1[0][i], '.3f'))
            elif i < 15:
                token.append(format(self.pred0[0][i], '.0f'))
            else:
                token.append(format(self.pred0[0][i], '.1f'))
        """
            #print(f'{self.targets[i]}: {token[i]}')


    def evaluate(self):
        np.set_printoptions(suppress=True)
        log = []
        for n in range(19):
            print(self.Y_test.columns[n], '\n')
            try:
                print('Linear Regression:')
                print(mean_absolute_error(self.pred0[:,n], self.Y_test.iloc[:,n]))
            except:
                pass

            try:
                print('Random Forest:')
                print(mean_absolute_error(self.pred1[:,n], self.Y_test.iloc[:,n]))
            except:
                pass

            try:
                print('Neural Network:')
                print(mean_absolute_error(self.pred2[:,n], self.Y_test.iloc[:,n]))
            except:
                pass
        
            try:
                print('Ensemble:')
                mean = np.mean([self.pred0[:,n], self.pred1[:,n], self.pred2[:,n]])
                print(mean - float(self.Y_test.iloc[0,n]))
                error = np.abs(mean - float(self.Y_test.iloc[0,n]))
                
                log.append(error)
            except:
                pass
            print('\n\n')
        print(np.mean(log))

In [18]:
m2 = Models('Josh Donaldson', 30, features_dict, targets_dict)
m2.wrangle()
m2.PMAPP_LR()
m2.PMAPP_RFR()
m2.PMAPP_NN(156)
m2.evaluate()
m2.show()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Train on 2133 samples, validate on 89 samples
Epoch 1/156
2133/2133 [==============================] - 2s 907us/sample - loss: 0.3985 - mean_absolute_percentage_error: 459.2109 - mean_absolute_error: 0.8102 - val_loss: 0.4202 - val_mean_absolute_percentage_error: 122.1970 - val_mean_absolute_error: 0.7981
Epoch 2/156
2133/2133 [==============================] - 2s 789us/sample - loss: 0.2991 - mean_absolute_percentage_error: 412.3047 - mean_absolute_error: 0.6764 - val_loss: 0.4167 - val_mean_absolute_percentage_error: 108.5620 - val_mean_absolute_error: 0.7868
Epoch 3/156
2133/2133 [==============================] - 2s 791us/sample - loss: 0.2661 - mean_absolute_percentage_error: 340.9402 - mean_absolute_error: 0.6278 - val_loss: 0.3844 - val_mean_absolute_percentage_error: 99.2134 - val_mean_absolute_error: 0.7423
Epoch 4/156
2133/2133 [==============================] - 2s 793us/sample - loss: 0.2475 - mean_absolute_percentage_error: 329.7390 - mean_absolute_error: 0.5968 - val_loss:

In [19]:
m1 = Models('Mike Moustakas', 29, features_dict, targets_dict)
m1.wrangle()
m1.PMAPP_LR()
m1.PMAPP_RFR()
#m1.Dummy()
m1.PMAPP_NN(21)
m1.evaluate()
m1.show()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Train on 2439 samples, validate on 102 samples
Epoch 1/21
2439/2439 [==============================] - 2s 888us/sample - loss: 0.4002 - mean_absolute_percentage_error: 435.3949 - mean_absolute_error: 0.8127 - val_loss: 0.3581 - val_mean_absolute_percentage_error: 185.0778 - val_mean_absolute_error: 0.7358
Epoch 2/21
2439/2439 [==============================] - 2s 770us/sample - loss: 0.2980 - mean_absolute_percentage_error: 336.0264 - mean_absolute_error: 0.6738 - val_loss: 0.3756 - val_mean_absolute_percentage_error: 165.8298 - val_mean_absolute_error: 0.7406
Epoch 3/21
2439/2439 [==============================] - 2s 799us/sample - loss: 0.2625 - mean_absolute_percentage_error: 312.4153 - mean_absolute_error: 0.6212 - val_loss: 0.3150 - val_mean_absolute_percentage_error: 176.0757 - val_mean_absolute_error: 0.6449
Epoch 4/21
2439/2439 [==============================] - 2s 766us/sample - loss: 0.2445 - mean_absolute_percentage_error: 288.0981 - mean_absolute_error: 0.5923 - val_loss: 0

In [20]:
m3 = Models('Mike Trout', 27, features_dict, targets_dict)
m3.wrangle()
m3.PMAPP_LR()
m3.PMAPP_RFR()
m3.PMAPP_NN(25)
m3.evaluate()
m3.show()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Train on 3051 samples, validate on 128 samples
Epoch 1/25
3051/3051 [==============================] - 3s 894us/sample - loss: 0.3903 - mean_absolute_percentage_error: 441.8583 - mean_absolute_error: 0.7946 - val_loss: 0.3343 - val_mean_absolute_percentage_error: 129.6173 - val_mean_absolute_error: 0.6761
Epoch 2/25
3051/3051 [==============================] - 2s 811us/sample - loss: 0.2970 - mean_absolute_percentage_error: 354.6111 - mean_absolute_error: 0.6682 - val_loss: 0.3639 - val_mean_absolute_percentage_error: 107.4237 - val_mean_absolute_error: 0.7223
Epoch 3/25
3051/3051 [==============================] - 2s 794us/sample - loss: 0.2675 - mean_absolute_percentage_error: 311.7105 - mean_absolute_error: 0.6249 - val_loss: 0.3314 - val_mean_absolute_percentage_error: 132.1074 - val_mean_absolute_error: 0.6670
Epoch 4/25
3051/3051 [==============================] - 2s 811us/sample - loss: 0.2475 - mean_absolute_percentage_error: 302.0820 - mean_absolute_error: 0.5932 - val_loss: 0